<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
</center>
<center>Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

# <center> Capstone проект №1. Идентификация пользователей по посещенным веб-страницам

В этом проекте мы будем решать задачу идентификации пользователя по его поведению в сети Интернет. Это сложная и интересная задача на стыке анализа данных и поведенческой психологии. В качестве примера, компания Яндекс решает задачу идентификации взломщика почтового ящика по его поведению. В двух словах, взломщик будет себя вести не так, как владелец ящика: он может не удалять сообщения сразу по прочтении, как это делал хозяин, он будет по-другому ставить флажки сообщениям и даже по-своему двигать мышкой. Тогда такого злоумышленника можно идентифицировать и "выкинуть" из почтового ящика, предложив хозяину войти по SMS-коду. Этот пилотный проект описан в [статье](https://habrahabr.ru/company/yandex/blog/230583/) на Хабрахабре. Похожие вещи делаются, например, в Google Analytics и описываются в научных статьях, найти можно многое по фразам "Traversal Pattern Mining" и "Sequential Pattern Mining".

<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

Мы будем решать похожую задачу: по последовательности из нескольких веб-сайтов, посещенных подряд один и тем же человеком, мы будем идентифицировать этого человека. Идея такая: пользователи Интернета по-разному переходят по ссылкам, и это может помогать их идентифицировать (кто-то сначала в почту, потом про футбол почитать, затем новости, контакт, потом наконец – работать, кто-то – сразу работать).

Будем использовать данные из [статьи](http://ceur-ws.org/Vol-1703/paper12.pdf) "A Tool for Classification of Sequential Data". И хотя мы не можем рекомендовать эту статью (описанные методы далеки от state-of-the-art, лучше обращаться к [книге](http://www.charuaggarwal.net/freqbook.pdf) "Frequent Pattern Mining" и последним статьям с ICDM), но данные там собраны аккуратно и представляют интерес.

Данные пришли с прокси-серверов Университета Блеза Паскаля и имеют очень простой вид. Для каждого пользователя заведен csv-файл с названием user\*\*\*\*.csv (где вместо звездочек – 4 цифры, соответствующие ID пользователя), а в нем посещения сайтов записаны в следующем формате: <br>

<center>*timestamp, посещенный веб-сайт*</center>

Скачать исходные данные можно по [ссылке](http://fc.isima.fr/~kahngi/cez13.zip) в статье, там же описание.
Для этого задания хватит данных не по всем 3000 пользователям, а по 10 и 150. [Ссылка](https://yadi.sk/d/3gscKIdN3BCASG) на архив *capstone_user_identification* (~7 Mb, в развернутом виде ~ 60 Mb). 

В финальном проекте уже придется столкнуться с тем, что не все операции можно выполнить за разумное время (скажем, перебрать с кросс-валидацией 100 комбинаций параметров случайного леса на этих данных вы вряд ли сможете), поэтому мы будем использовать параллельно 2 выборки: по 10 пользователям и по 150. Для 10 пользователей будем писать и отлаживать код, для 150 – будет рабочая версия. 

Данные устроены следующем образом:

 - В каталоге `10users` лежат 10 csv-файлов с названием вида "user[USER_ID].csv", где [USER_ID] – ID пользователя;
 - Аналогично для каталога `150users` – там 150 файлов;
 - В `3users` – игрушечный пример из 3 файлов, это для отладки кода предобработки, который вы далее напишете.

На 5 неделе будет задание по [соревнованию](https://inclass.kaggle.com/c/identify-me-if-you-can4) Kaggle Inclass, которое организовано специально под Capstone проект нашей специализации. Соревнование уже открыто и, конечно, желающие могут начать уже сейчас.

# <center>Неделя 1. Подготовка данных к анализу и построению моделей

Первая часть проекта посвящена подготовке данных для дальнейшего описательного анализа и построения прогнозных моделей. Надо будет написать код для предобработки данных (исходно посещенные веб-сайты указаны для каждого пользователя в отдельном файле) и формирования единой обучающей выборки. Также в этой части мы познакомимся с разреженным форматом данных (матрицы Scipy.sparse), который хорошо подходит для данной задачи. 

**План 1 недели:**
 - Часть 1. Подготовка обучающей выборки
 - Часть 2. Работа с разреженным форматом данных

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций 1 и 2 недели курса "Математика и Python для анализа данных":**
   - [Циклы, функции, генераторы, list comprehension](https://www.coursera.org/learn/mathematics-and-python/lecture/Kd7dL/tsikly-funktsii-ghienieratory-list-comprehension)
   - [Чтение данных из файлов](https://www.coursera.org/learn/mathematics-and-python/lecture/8Xvwp/chtieniie-dannykh-iz-failov)
   - [Запись файлов, изменение файлов](https://www.coursera.org/learn/mathematics-and-python/lecture/vde7k/zapis-failov-izmienieniie-failov)
   - [Pandas.DataFrame](https://www.coursera.org/learn/mathematics-and-python/lecture/rcjAW/pandas-data-frame)
   - [Pandas. Индексация и селекция](https://www.coursera.org/learn/mathematics-and-python/lecture/lsXAR/pandas-indieksatsiia-i-sieliektsiia)
   
**Кроме того, в задании будут использоваться библиотеки Python [glob](https://docs.python.org/3/library/glob.html), [pickle](https://docs.python.org/2/library/pickle.html) и класс [csr_matrix](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.sparse.csr_matrix.html) из scipy.sparse.**

Наконец, для лучшей воспроизводимости результатов приведем список версий основных используемых в проекте библиотек: NumPy, SciPy, Pandas, Matplotlib, Statsmodels и Scikit-learn. Для этого воспользуемся расширением [watermark](https://github.com/rasbt/watermark).

In [1]:
#!pip install watermark
#%load_ext watermark

In [2]:
%watermark -v -m -p numpy,scipy,pandas,matplotlib,statsmodels,sklearn -g

ERROR:root:Line magic function `%watermark` not found.


In [3]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import os
import pickle
# pip install tqdm
from tqdm import tqdm 
import numpy as np
import pandas as pd
import itertools
from scipy.sparse import csr_matrix

**Посмотрим на один из файлов с данными о посещенных пользователем (номер 31) веб-страницах.**

In [8]:
!pwd

/tmp/working/machinlearning/6/ident/week1


In [9]:
# Поменяйте на свой путь к данным
# PATH_TO_DATA = '/tmp/working/machinleaning/ml/6/ident/data/capstone_user' 
PATH_TO_DATA = '/tmp/working/machinlearning/6/ident/data/capstone_user/' 

In [10]:
f_name=os.path.join(PATH_TO_DATA, 
                                       '10users/user0031.csv')

In [12]:
os.listdir(PATH_TO_DATA)

['3users', '10users', '150users']

In [13]:
user31_data = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                       '10users/user0031.csv'))

In [14]:
user31_data.head()

,timestamp,site
0,2013-11-15 08:12:07,fpdownload2.macromedia.com
1,2013-11-15 08:12:17,laposte.net
2,2013-11-15 08:12:17,www.laposte.net
3,2013-11-15 08:12:17,www.google.com
4,2013-11-15 08:12:18,www.laposte.net


**Поставим задачу классификации: идентифицировать пользователя по сессии из 10 подряд посещенных сайтов. Объектом в этой задаче будет сессия из 10 сайтов, последовательно посещенных одним и тем же пользователем, признаками – индексы этих 10 сайтов (чуть позже здесь появится "мешок" сайтов, подход Bag of Words). Целевым классом будет id пользователя.**

### <center>Пример для иллюстрации</center>
**Пусть пользователя всего 2, длина сессии – 2 сайта.**

<center>user0001.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">00:00:01</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-yw4l">00:00:11</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:16</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:20</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

<center>user0002.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">00:00:02</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
  <tr>
    <td class="tg-yw4l">00:00:14</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:17</td>
    <td class="tg-031e">facebook.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:25</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

Идем по 1 файлу, нумеруем сайты подряд: vk.com – 1, google.com – 2 и т.д. Далее по второму файлу. 

Отображение сайтов в их индесы должно получиться таким:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">site</th>
    <th class="tg-yw4l">site_id</th>
  </tr>
  <tr>
    <td class="tg-yw4l">vk.com</td>
    <td class="tg-yw4l">1</td>
  </tr>
  <tr>
    <td class="tg-yw4l">google.com</td>
    <td class="tg-yw4l">2</td>
  </tr>
  <tr>
    <td class="tg-yw4l">yandex.ru</td>
    <td class="tg-yw4l">3</td>
  </tr>
  <tr>
    <td class="tg-yw4l">facebook.com</td>
    <td class="tg-yw4l">4</td>
  </tr>
</table>

Тогда обучающая выборка будет такой (целевой признак – user_id):
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-s6z2{text-align:center}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-hgcj{font-weight:bold;text-align:center}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-hgcj">session_id</th>
    <th class="tg-hgcj">site1</th>
    <th class="tg-hgcj">site2</th>
    <th class="tg-amwm">user_id</th>
  </tr>
  <tr>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">2</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">2</td>
  </tr>
  <tr>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">2</td>
  </tr>
</table>

Здесь 1 объект – это сессия из 2 посещенных сайтов 1-ым пользователем (target=1). Это сайты vk.com и google.com (номер 1 и 2). И так далее, всего 4 сессии. Пока сессии у нас не пересекаются по сайтам, то есть посещение каждого отдельного сайта относится только к одной сессии.

## Часть 1. Подготовка обучающей выборки
Реализуйте функцию *prepare_train_set*, которая принимает на вход путь к каталогу с csv-файлами *path_to_csv_files* и параметр *session_length* – длину сессии, а возвращает 2 объекта:
- DataFrame, в котором строки соответствуют уникальным сессиям из *session_length* сайтов, *session_length* столбцов – индексам этих *session_length* сайтов и последний столбец – ID пользователя
- частотный словарь сайтов вида {'site_string': [site_id, site_freq]}, например для недавнего игрушечного примера это будет {'vk.com': (1, 2), 'google.com': (2, 2), 'yandex.ru': (3, 3), 'facebook.com': (4, 1)}

Детали:
- Смотрите чуть ниже пример вывода, что должна возвращать функция
- Используйте glob (или аналоги) для обхода файлов в каталоге. Для определенности, отсортируйте список файлов лексикографически. Удобно использовать `tqdm` для отслеживания числа выполненных итераций цикла
- Создайте частотный словарь уникальных сайтов (вида {'site_string': (site_id, site_freq)}) и заполняйте его по ходу чтения файлов. Начните с 1
- Рекомендуется меньшие индексы давать более часто попадающимся сайтам (приницип наименьшего описания)
- Не делайте entity recognition, считайте *google.com*, *http://www.google.com* и *www.google.com* разными сайтами (подключить entity recognition можно уже в рамках индивидуальной работы над проектом)
- Скорее всего в файле число записей не кратно числу *session_length*. Тогда последняя сессия будет короче. Остаток заполняйте нулями. То есть если в файле 24 записи и сессии длины 10, то 3 сессия будет состоять из 4 сайтов, и ей мы сопоставим вектор [*site1_id*, *site2_id*, *site3_id*, *site4_id*, 0, 0, 0, 0, 0, 0, *user_id*] 
- В итоге некоторые сессии могут повторяться – оставьте как есть, не удаляйте дубликаты. Если в двух сессиях все сайты одинаковы, но сессии принадлежат разным пользователям, то тоже оставляйте как есть, это естественная неопределенность в данных
- Не оставляйте в частотном словаре сайт 0 (уже в конце, когда функция возвращает этот словарь)
- 150 файлов из *capstone_websites_data/150users/* у меня обработались за 1.7 секунды, но многое, конечно, зависит от реализации функции и от используемого железа. И вообще, первая реализация скорее всего будет не самой эффективной, дальше можно заняться профилированием (особенно если планируете запускать этот код для 3000 пользователей). Также эффективная реализация этой функции поможет нам на следующей неделе.

In [15]:
from collections import OrderedDict

In [16]:
for file in sorted(glob(PATH_TO_DATA+'/3users/*')):
    print(file)

/tmp/working/machinlearning/6/ident/data/capstone_user//3users/user0001.csv
/tmp/working/machinlearning/6/ident/data/capstone_user//3users/user0002.csv
/tmp/working/machinlearning/6/ident/data/capstone_user//3users/user0003.csv


In [19]:
#Split in_to session
def split(arr,num):
    len_=len(arr)
    num_iter = int(np.ceil(len(arr)/num))
    r = num_iter * num - len_

    arr = np.concatenate((np.array(arr),np.array([0]*r,dtype=np.int16)))
    return np.split(arr,num_iter)

def prepare_train_set(path_to_csv_files, session_length=10):
    ''' ВАШ КОД ЗДЕСЬ '''
    ##Create_dict
    path_to_csv_files +='/*'
    users_data = (pd.read_csv(f) for f in glob(path_to_csv_files))
    concatenated_df = pd.concat(users_data, ignore_index=True)
    unique_val = concatenated_df.site.value_counts()

    frame = pd.DataFrame(unique_val)
    size = frame.shape[0]
    frame['inx'] = np.arange(1,size+1)

    dict_site=dict(\
    list(\
        zip(frame.index.values,\
            zip(frame['inx'].values,frame['site'].values))))
    
    
    lst = []
#     from IPython.core.debugger import Tracer; Tracer()() 
    for u_id,f in tqdm(enumerate(sorted(glob(path_to_csv_files)))):
        user_data = pd.read_csv(f)
        list_ = list(\
        map(lambda x: dict_site[x][0],user_data.site.values))
#         print(list_)
        list_ = list(map(lambda x: np.append(x,u_id+1),split(list_,10)))
        
        lst.append(list_)
#     print(lst)
    return pd.DataFrame(data =list(itertools.chain(*lst)),\
                        columns=['ind{0}'.format(i)\
                                 for i in range(1,11)]+['user_id']),\
    dict_site

In [21]:
train_data_toy, site_freq_3users = prepare_train_set(PATH_TO_DATA+'/3users')

3it [00:00, 250.59it/s]


In [22]:
train_data_toy

,ind1,ind2,ind3,ind4,ind5,ind6,ind7,ind8,ind9,ind10,user_id
0,4,2,2,11,2,1,8,6,7,10,1
1,4,1,1,1,0,0,0,0,0,0,1
2,4,2,5,5,2,0,0,0,0,0,2
3,3,1,2,1,2,1,1,6,9,3,3
4,3,1,2,0,0,0,0,0,0,0,3


**Примените полученную функцию к игрушечному примеру, убедитесь, что все работает как надо.**

In [23]:
!cat $PATH_TO_DATA/3users/user0001.csv

timestamp,site
2013-11-15 09:28:17,vk.com
2013-11-15 09:33:04,oracle.com
2013-11-15 09:52:48,oracle.com
2013-11-15 11:37:26,geo.mozilla.org
2013-11-15 11:40:32,oracle.com
2013-11-15 11:40:34,google.com
2013-11-15 11:40:35,accounts.google.com
2013-11-15 11:40:37,mail.google.com
2013-11-15 11:40:40,apis.google.com
2013-11-15 11:41:35,plus.google.com
2013-11-15 12:40:35,vk.com
2013-11-15 12:40:37,google.com
2013-11-15 12:40:40,google.com
2013-11-15 12:41:35,google.com


In [24]:
!cat $PATH_TO_DATA/3users/user0002.csv

timestamp,site
2013-11-15 09:28:17,vk.com
2013-11-15 09:33:04,oracle.com
2013-11-15 09:52:48,football.kulichki.ru
2013-11-15 11:37:26,football.kulichki.ru
2013-11-15 11:40:32,oracle.com


In [25]:
!cat $PATH_TO_DATA/3users/user0003.csv

timestamp,site
2013-11-15 09:28:17,meduza.io
2013-11-15 09:33:04,google.com
2013-11-15 09:52:48,oracle.com
2013-11-15 11:37:26,google.com
2013-11-15 11:40:32,oracle.com
2013-11-15 11:40:34,google.com
2013-11-15 11:40:35,google.com
2013-11-15 11:40:37,mail.google.com
2013-11-15 11:40:40,yandex.ru
2013-11-15 11:41:35,meduza.io
2013-11-15 12:28:17,meduza.io
2013-11-15 12:33:04,google.com
2013-11-15 12:52:48,oracle.com


In [26]:
train_data_toy, site_freq_3users = prepare_train_set(os.path.join(PATH_TO_DATA, '3users'), 
                                                     session_length=10)

3it [00:00, 375.37it/s]


In [27]:
train_data_toy

,ind1,ind2,ind3,ind4,ind5,ind6,ind7,ind8,ind9,ind10,user_id
0,4,2,2,11,2,1,8,6,7,10,1
1,4,1,1,1,0,0,0,0,0,0,1
2,4,2,5,5,2,0,0,0,0,0,2
3,3,1,2,1,2,1,1,6,9,3,3
4,3,1,2,0,0,0,0,0,0,0,3


Частоты сайтов (второй элемент кортежа) точно должны быть такими, нумерация может быть любой (первые элементы кортежей могут отличаться).

**1. Примените полученную функцию к данным по 10 пользователям, посмотрите, сколько уникальных сессий из 10 сайтов получится, запишите ответ в файл с помощью функции *write_answer_to_file* – это будет ответом к первому вопросу теста. Если ответ получился неправильный, значит где-то ошибка в функции *prepare_train_set*.**

In [28]:
train_data_10users, site_freq_10users =  prepare_train_set(os.path.join(PATH_TO_DATA, '10users'), 
                                                     session_length=10)

10it [00:00, 36.97it/s]


In [29]:
train_data_10users.shape

(14061, 11)

In [30]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

In [31]:
write_answer_to_file(14061,\
                     'answer1_1.txt')

In [32]:
len(site_freq_10users)

4913

**2. Сколько всего уникальных сайтов в выборке из 10 пользователей? **

In [33]:
write_answer_to_file(4913, 
                     'answer1_2.txt')

**3. Примените полученную функцию к данным по 150 пользователям, посмотрите, сколько уникальных сессий из 10 сайтов получится, запишите ответ в файл с помощью функции *write_answer_to_file* – это будет ответом к третьему вопросу. Если ответ получился неправильный, значит где-то ошибка в функции *prepare_train_set*.**

In [34]:
%%time
train_data_150users, site_freq_150users =prepare_train_set(os.path.join(PATH_TO_DATA, '150users'), 
                                                     session_length=10)

150it [00:02, 63.64it/s]


CPU times: user 4.74 s, sys: 152 ms, total: 4.89 s
Wall time: 5.67 s


In [35]:
train_data_150users.shape

(137019, 11)

In [36]:
write_answer_to_file(137019, 
                     'answer1_3.txt')

**4. Сколько всего уникальных сайтов в выборке из 150 пользователей? **

In [37]:
len(site_freq_150users)

27797

In [38]:
write_answer_to_file(27797, 
                     'answer1_4.txt')

**5. Выведите топ-10 самых популярных сайтов среди посещенных 150 пользователями. Запишите ответ в файл *answer1_5.txt* через пробел – все 10 сайтов на одной строке.**

In [39]:
import heapq

In [40]:
top10_popular = heapq.nlargest(10,site_freq_150users.items(),key= lambda x: x[1][1])

In [41]:
list_ans = [i[0] for i in top10_popular]
write_answer_to_file(" ".join(list_ans), 
                     'answer1_5.txt')

In [42]:
!cat answer1_5.txt

www.google.fr www.google.com www.facebook.com apis.google.com s.youtube.com clients1.google.com mail.google.com plus.google.com safebrowsing-cache.google.com www.youtube.com

**Для дальнейшего анализа запишем полученные объекты DataFrame в csv-файлы.**

In [43]:
train_data_10users.to_csv(os.path.join(PATH_TO_DATA, 
                                       'train_data_10users.csv'), 
                        index_label='session_id', float_format='%d')
train_data_150users.to_csv(os.path.join(PATH_TO_DATA, 
                                        'train_data_150users.csv'), 
                         index_label='session_id', float_format='%d')

## Часть 2. Работа с разреженным форматом данных


**Если так подумать, то полученные признаки *site1*, ..., *site10* смысла не имеют как признаки в задаче классификации. А вот если воспользоваться идеей мешка слов из анализа текстов – это другое дело. Создадим новые матрицы, в которых строкам будут соответствовать сессии из 10 сайтов, а столбцам – индексы сайтов. На пересечении строки $i$ и столбца $j$ будет стоять число $n_{ij}$ – cколько раз сайт $j$ встретился в сессии номер $i$. Делать это будем с помощью разреженных матриц Scipy – [csr_matrix](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.sparse.csr_matrix.html). Прочитайте документацию, разберитесь, как использовать разреженные матрицы и создайте такие матрицы для наших данных. Сначала проверьте на игрушечном примере, затем примените для 10 и 150 пользователей. **

**Обратите внимание, что в коротких сессиях, меньше 10 сайтов, у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить.** 

In [44]:
from collections import Counter, OrderedDict

class OrderedCounter(Counter, OrderedDict):
    def __repr__(self):
        return '%s(%r)' % (self.__class__.__name__,
                        OrderedDict(self))
    def __reduce__(self):
        return self.__class__, (OrderedDict(self),)

In [45]:
X_toy, y_toy = train_data_toy.iloc[:, :-1].values, train_data_toy.iloc[:, -1].values

In [46]:
def to_sparce(X,n):
    indptr = [0]
    count = 0
    indices = []
    data = []
    for col in tqdm(range(X.shape[0])):
        counter = Counter(X[col])
        count += len(counter)
        indptr.append(count)
        indices += counter.keys()
        data += counter.values()

    return csr_matrix((data, indices, indptr), shape=(X.shape[0], n))[:,1:]

In [47]:
''' ВАШ КОД ЗДЕСЬ '''   
X_sparse_toy = to_sparce(X_toy,12)

100%|██████████| 5/5 [00:00<00:00, 3956.14it/s]


In [48]:
X_sparse_toy.todense()

matrix([[1, 3, 0, 1, 0, 1, 1, 1, 0, 1, 1],
        [3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0],
        [4, 2, 2, 0, 0, 1, 0, 0, 1, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

**Размерность разреженной матрицы должна получиться равной 11, поскольку в игрушечном примере 3 пользователя посетили 11 уникальных сайтов.**

In [49]:
X_10users, y_10users = train_data_10users.iloc[:, :-1].values, \
                       train_data_10users.iloc[:, -1].values
X_150users, y_150users = train_data_150users.iloc[:, :-1].values, \
                         train_data_150users.iloc[:, -1].values

In [50]:
X_sparse_10users =to_sparce(X_10users,4913+1) #''' ВАШ КОД ЗДЕСЬ '''
X_sparse_150users =to_sparce(X_150users,27797+1) #''' ВАШ КОД ЗДЕСЬ '''

100%|██████████| 137019/137019 [00:01<00:00, 97686.63it/s]


In [51]:
X_10users

array([[ 193,  578,  134, ...,  134,  204,  134],
       [ 424,  192,  687, ..., 4669,  220,   55],
       [  55,    3,   55, ...,  334,  921,   55],
       ..., 
       [  33,    4,    9, ...,   33,    4,    4],
       [   4,    4,    4, ...,    3,    9,    4],
       [  10,   21,    0, ...,    0,    0,    0]])

In [52]:
y_10users.shape

(14061,)

**Сохраним эти разреженные матрицы с помощью [pickle](https://docs.python.org/2/library/pickle.html) (сериализация в Python), также сохраним вектора *y_10users, y_150users* – целевые значения (id пользователя)  в выборках из 10 и 150 пользователей. То что названия этих матриц начинаются с X и y, намекает на то, что на этих данных мы будем проверять первые модели классификации.
Наконец, сохраним также и частотные словари сайтов для 3, 10 и 150 пользователей.**

In [53]:
with open(os.path.join(PATH_TO_DATA, 
                       'X_sparse_10users.pkl'), 'wb') as X10_pkl:
    pickle.dump(X_sparse_10users, X10_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 
                       'y_10users.pkl'), 'wb') as y10_pkl:
    pickle.dump(y_10users, y10_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 
                       'X_sparse_150users.pkl'), 'wb') as X150_pkl:
    pickle.dump(X_sparse_150users, X150_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 
                       'y_150users.pkl'), 'wb') as y150_pkl:
    pickle.dump(y_150users, y150_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 
                       'site_freq_3users.pkl'), 'wb') as site_freq_3users_pkl:
    pickle.dump(site_freq_3users, site_freq_3users_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 
                       'site_freq_10users.pkl'), 'wb') as site_freq_10users_pkl:
    pickle.dump(site_freq_10users, site_freq_10users_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 
                       'site_freq_150users.pkl'), 'wb') as site_freq_150users_pkl:
    pickle.dump(site_freq_150users, site_freq_150users_pkl, protocol=2)

**Чисто для подстраховки проверим, что число столбцов в разреженных матрицах `X_sparse_10users` и `X_sparse_150users` равно ранее посчитанным числам уникальных сайтов для 10 и 150 пользователей соответственно.**

In [54]:
assert X_sparse_10users.shape[1] == len(site_freq_10users)

In [55]:
assert X_sparse_150users.shape[1] == len(site_freq_150users)

In [56]:
!gcc -v

Using built-in specs.
COLLECT_GCC=gcc
COLLECT_LTO_WRAPPER=/usr/lib/gcc/x86_64-linux-gnu/4.9/lto-wrapper
Target: x86_64-linux-gnu
Configured with: ../src/configure -v --with-pkgversion='Debian 4.9.2-10' --with-bugurl=file:///usr/share/doc/gcc-4.9/README.Bugs --enable-languages=c,c++,java,go,d,fortran,objc,obj-c++ --prefix=/usr --program-suffix=-4.9 --enable-shared --enable-linker-build-id --libexecdir=/usr/lib --without-included-gettext --enable-threads=posix --with-gxx-include-dir=/usr/include/c++/4.9 --libdir=/usr/lib --enable-nls --with-sysroot=/ --enable-clocale=gnu --enable-libstdcxx-debug --enable-libstdcxx-time=yes --enable-gnu-unique-object --disable-vtable-verify --enable-plugin --with-system-zlib --disable-browser-plugin --enable-java-awt=gtk --enable-gtk-cairo --with-java-home=/usr/lib/jvm/java-1.5.0-gcj-4.9-amd64/jre --enable-java-home --with-jvm-root-dir=/usr/lib/jvm/java-1.5.0-gcj-4.9-amd64 --with-jvm-jar-dir=/usr/lib/jvm-exports/java-1.5.0-gcj-4.9-amd64 --with-arch-direct

## Пути улучшения
В этом проекте свобода творчества на каждом шаге, а 7 неделя проекта посвящена общему описанию (`html`, `ipynb` или `pdf`) и взаимному оцениванию проектов. Что еще можно добавить по первой части проекта:
-  можно обработать исходные данные по 3000 пользователей; обучать на такой выборке модели лучше при наличии доступа к хорошим мощностям (можно арендовать инстанс Amazon EC2, как именно, описано [тут](https://habrahabr.ru/post/280562/)). Хотя далее в курсе мы познакомимся с алгоритмами, способными обучаться на больших выборках при малых вычислительных потребностях;
- Помимо явного создания разреженного формата можно еще составить выборки с помощью `CountVectorizer`, `TfidfVectorizer` и т.п. Поскольку данные по сути могут быть описаны как последовательности, то можно вычислять n-граммы сайтов. Работает все это или нет, мы будем проверять в [соревновании](https://inclass.kaggle.com/c/identify-me-if-you-can4) Kaggle Inclass (желающие могут начать уже сейчас).

На следующей неделе мы еще немного поготовим данные и потестируем первые гипотезы, связанные с нашими наблюдениями. 